This notebook follows 04_extract_data_from_supplementary_excel_files.

Inputs:
1. NIHMS687993-supplement-supp_data_4.xlsx was downloaded from [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4481139/).

Outputs:
1. FINAL_MARKERS_FOR_EACH_CLUSTER.csv

In [1]:
import pandas as pd
import numpy as np
import itertools
from harrison_functions.formatting.list_tools import pairwise

In [2]:
to_save=False

# Cluster Markers

In [3]:
cluster_markers = pd.read_excel('data/downloads/NIHMS687993-supplement-supp_data_4.xlsx',
                                sheet_name='FINAL_MARKERS_FOR_EACH_CLUSTER.',
                                header=None, names=['gene_symbol', 'myAUC', 'myDiff', 'power', 'cluster_no'])
cluster_markers = cluster_markers.rename(columns={"Unnamed: 0": 'gene_symbol'})
print(cluster_markers.shape)

(4296, 5)


In [4]:
table_start_idx = cluster_markers[cluster_markers['gene_symbol'] == 'cluster no.'].index.to_list()
table_start_end = [(i+2, j-2) for i, j in pairwise(table_start_idx)]  # get start and end indices for each subtable

In [7]:
# identify tables that were corrupted
needs_cleaning = [18,  # type1
                  20, 21, 30, 31,  # type2
                  24, 27, 33]  # type3

df_list = [cluster_markers[i: j] for i, j in table_start_end]  # split df into multiple tables

# clean up tables

for i in [18]:
    df_list[i-1]['cluster_no'] = df_list[i-1]['cluster_no'].apply(lambda x: x.split(' ')[0])
    df_list[i-1]['power'] = df_list[i-1].apply(lambda df: df['myDiff'].split(' ')[-1]+df['power'].split(' ')[0], axis=1)
    df_list[i-1]['myDiff'] = df_list[i-1].apply(lambda df: df['myAUC'].split(' ')[-1]+df['myDiff'].split(' ')[0], axis=1)
    df_list[i-1]['myAUC'] = df_list[i-1].apply(lambda df: df['gene_symbol'].split(' ')[-1]+df['myAUC'].split(' ')[0], axis=1)
    df_list[i-1]['gene_symbol'] = df_list[i-1]['gene_symbol'].apply(lambda x: x.split(' ')[0])

for i in [20, 21, 30, 31]:
    df_list[i-1]['cluster_no'] = df_list[i-1]['power']
    df_list[i-1]['power'] = df_list[i-1]['myDiff'].apply(lambda x: x.split(' ')[-1])
    df_list[i-1]['myDiff'] = df_list[i-1].apply(lambda df: df['myAUC']+df['myDiff'].split(' ')[0], axis=1)
    df_list[i-1]['myAUC'] = df_list[i-1]['gene_symbol'].apply(lambda x: x.split(' ')[-1])
    df_list[i-1]['gene_symbol'] = df_list[i-1]['gene_symbol'].apply(lambda x: x.split(' ')[0])
    
for i in [24, 27, 33]:
    df_list[i-1]['cluster_no'] = df_list[i-1].apply(lambda df: df['power'].split(' ')[-1]+df['cluster_no'].split(' ')[0], axis=1)
    df_list[i-1]['power'] = df_list[i-1].apply(lambda df: df['myDiff'].split(' ')[-1]+df['power'].split(' ')[0], axis=1)
    df_list[i-1]['myDiff'] = df_list[i-1].apply(lambda df: df['myAUC']+df['myDiff'].split(' ')[0], axis=1)
    df_list[i-1]['myAUC'] = df_list[i-1].apply(lambda df: df['gene_symbol'].split(' ')[-1], axis=1)
    df_list[i-1]['gene_symbol'] = df_list[i-1]['gene_symbol'].apply(lambda x: x.split(' ')[0])

/home/harrisonized/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/harrisonized/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/harrisonized/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [8]:
# rejoin the tables
cluster_markers = pd.concat(df_list)

In [10]:
cluster_markers['gene_symbol'] = cluster_markers['gene_symbol'].astype(str)
cluster_markers['myAUC'] = cluster_markers['myAUC'].astype(float)
cluster_markers['myDiff'] = cluster_markers['myDiff'].astype(float)
cluster_markers['power'] = cluster_markers['power'].astype(float)
cluster_markers['cluster_no'] = cluster_markers['cluster_no'].astype(int)
cluster_markers

,gene_symbol,myAUC,myDiff,power,cluster_no
4,CALB1,0.966,3.615047,0.466,1
5,SLC4A3,0.963,3.448571,0.463,1
6,TPM3,0.965,3.151521,0.465,1
7,SEPT4,0.964,2.939258,0.464,1
8,VIM,0.944,2.937992,0.444,1
...,...,...,...,...,...
4134,SAG,0.140,-2.497791,0.360,38
4135,PDE6B,0.194,-2.533895,0.306,38
4136,PRPH2,0.151,-2.581111,0.349,38
4137,ROM1,0.175,-2.590215,0.325,38


In [11]:
if to_save:
    cluster_markers.to_csv('data/FINAL_MARKERS_FOR_EACH_CLUSTER.csv', index=None)